In [12]:
import os
import shutil as sh
import zipfile as zf
import matplotlib.pyplot as plt
import numpy as np
import pydicom as dicom

In [13]:
DATA_SRC_PATH = 'C:\\Projects\\AIUniversity\\Segments'
DATA_DST_PATH = 'C:\\Projects\\AIUniversity\\Dataset'

In [14]:
def path_has_subdir(path):
  items = [item for item in os.listdir(path) if os.path.isdir(f'{path}\\{item}')]
  return len(items)!=0

In [26]:
def get_dir_list(path: str, check = lambda item: True) -> list[str]:
  items = [{'path': path, 'item': item, 'fullpath': f'{path}\\{item}'} \
           for item in os.listdir(path) if check(f'{path}\\{item}')]
  return items


In [36]:
def find_leaf_dir_list(path):
  dirs = []
  items = get_dir_list(path, lambda path: os.path.isdir(path))
  for item in items:
    if not path_has_subdir(item['fullpath']):
      dirs.append(item)
    else:
      dirs += find_leaf_dir_list(item['fullpath'])
  return dirs


In [79]:
def get_dst_dir_name(path_dict):
  items = get_dir_list(path_dict['fullpath'], \
                       lambda path: os.path.isfile(path) and '.dcm' in path)
  if 'Segmentation' in path_dict['item']:
    return 'Segmentation'
  elif len(items)==1 and items[0]['item']=='1-1.dcm':
    return 'Throwble'
  elif len(items)>0:
    return 'Source'
  else:
    return path_dict['item']

In [72]:
def create_copy(path_dict, dst_path):
  newpath = f'{dst_path}\\{get_dst_dir_name(path_dict)}'
  os.makedirs(newpath)
  items = get_dir_list(path_dict['fullpath'], lambda path: os.path.isfile(path))
  for item in items:
    sh.copy(item['fullpath'], f"{newpath}\\{item['item']}")
  return newpath

In [81]:
if os.path.exists(DATA_DST_PATH): sh.rmtree(DATA_DST_PATH)
os.makedirs(DATA_DST_PATH)
path_dict = get_dir_list(DATA_SRC_PATH, lambda item: os.path.isdir(item))
for item in path_dict:
  newpath = create_copy(item, DATA_DST_PATH)
  paths = find_leaf_dir_list(item['fullpath'])
  for path in paths:
    create_copy(path, newpath)
